In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta
from glob import glob

In [2]:
files = glob('edu.my/*.html')
len(files)

219

In [3]:
all_a = []
for f in tqdm(files):
    with open(f) as fopen:
        data = fopen.read()
    soup = BeautifulSoup(data, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href', '') for a_ in a if a_.get('href', '').endswith('.pdf')]
    all_a.extend(a)

100%|█████████████████████████████████████████| 219/219 [00:25<00:00,  8.72it/s]


In [8]:
all_a = list(set(all_a))
len(all_a)

27827

In [9]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/5aa5257608b61e8fcc828e99fbd070d5ca7358e3/mp.py

In [10]:
import mp

In [11]:
with open('all_a.json', 'w') as fopen:
    json.dump(all_a, fopen)

In [12]:
all_a_f = [(no, f) for no, f in enumerate(all_a)]

In [13]:
!mkdir pdf

In [14]:
def loop(files):
    files, _ = files
    for f in tqdm(files):
        i, f = f
        file = os.path.join('pdf', f'{i}.pdf')
        if os.path.exists(file):
            continue
        
        try:
            r = requests.get(f, timeout = 60)
            with open(file, 'wb') as fopen:
                fopen.write(r.content)
        except:
            pass

In [15]:
loop((all_a_f[:2], 0))

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  3.74it/s]


In [16]:
mp.multiprocessing(all_a_f, loop, cores = 10, returned = False)

 80%|███████████████████████████████▎       | 2232/2782 [50:36<12:03,  1.32s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

